In [16]:
import asyncio

import gradio as gr
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from autogen_core import CancellationToken
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.mcp import (
    StreamableHttpServerParams,
    mcp_server_tools,
)
from dotenv import load_dotenv

: 

In [10]:
load_dotenv();

In [11]:
server = StreamableHttpServerParams(
    url="http://127.0.0.1:8000/mcp",
)

In [12]:
tools = await mcp_server_tools(server)

In [13]:
model = OpenAIChatCompletionClient(model="gpt-4o-mini")
agent = AssistantAgent(
    name="app",
    model_client=model,
    tools=tools,
    system_message=(
        "You are a helpful app."
    ),
    reflect_on_tool_use=True
)

In [ ]:
async def chat_fn(user_message, chat_history):
    result = await agent.run(task=user_message)  # handles tools internally
    reply = ""
    for m in reversed(result.messages):
        if isinstance(m, TextMessage) and m.source == agent.name:
            reply = m.content
            break
    chat_history.append((user_message, reply))
    return "", chat_history

# --- reset clears the agent's internal conversation state ---
async def reset_fn():
    await agent.on_reset(CancellationToken())
    return gr.update(value=[])

with gr.Blocks(title="AutoGen Test Chat") as demo:
    gr.Markdown("### AutoGen Test Chat")
    chat = gr.Chatbot(height=500)
    msg = gr.Textbox(placeholder="Type a message and press Enter...")
    with gr.Row():
        reset = gr.Button("Reset conversation", variant="secondary")
    msg.submit(chat_fn, [msg, chat], [msg, chat])
    reset.click(reset_fn, outputs=chat)

demo.queue().launch()